In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
%run imports.py

In [40]:
from goexplore import *

In [41]:
import glob

In [42]:
!pwd

/app/atari


In [43]:
archive = np.load('/app/atari/data/ge_specialist/ge_specialist_Alien_0000.npy', allow_pickle=True).item()

In [44]:
traj = archive['traj']
ret = archive['ret']
novelty = archive['novelty']
is_leaf = archive['is_leaf']

In [48]:
is_leaf.sum()/len(is_leaf)

0.015143062086554556

In [ ]:
len(archive['trajs'])

In [48]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.data = None
        
    def insert_one(self, word, data):
        node = self
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.data = data
        
    def insert_all(self, words, datas):
        for word, data in zip(words, datas):
            self.insert_one(word, data)
        
    def trie_num_nodes(root):
        assert root is not None
        count = 0
        stack = [root]
        while stack:
            node = stack.pop()
            count += 1
            stack.extend(node.children.values())
        return count

def filter_substrings(trajs):
    root = TrieNode()
    root.insert_all(trajs, np.arange(len(trajs)))
    
    ids = []
    nodes_to_visit = [root]
    while len(nodes_to_visit)>0:
        node = nodes_to_visit.pop(0)
        nodes_to_visit.extend(node.children.values())
        if len(node.children)==0:
            assert node.data is not None
            ids.append(node.data)
    ids = np.array(ids)
    is_leaf = np.zeros(len(trajs), dtype=bool)
    is_leaf[ids] = True
    return is_leaf, ids



In [13]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.data = None
        
class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.nodes = [self.root]
        
    def insert_one(self, word, data=None):
        node = self.root
        for char in word:
            if char not in node.children:
                self.nodes.append(TrieNode())
                node.children[char] = len(self.nodes)-1
            node = self.nodes[node.children[char]]
        node.data = data
        
    def insert_all(self, words, datas=None):
        for i in range(len(words)):
            self.insert_one(words[i], datas[i] if datas is not None else None)

In [25]:
mydir = '/data/vision/phillipi/akumar01/exploration-generalization/atari/data/'
for name in tqdm(os.listdir(mydir+'/ge_specialist')):
    f = mydir+'/ge_specialist/'+name
    archive = np.load(f, allow_pickle=True).item()

    trie = Trie()
    trie.insert_all(archive['trajs'], [None for _ in range(len(archive['trajs']))])
    
    os.makedirs(mydir+'/temp/raw/', exist_ok=True)
    os.makedirs(mydir+'/temp/tri/', exist_ok=True)
    np.save(mydir+'/temp/raw/'+name, archive['trajs'])
    np.save(mydir+'/temp/tri/'+name, trie)

  0%|          | 0/137 [00:00<?, ?it/s]

In [15]:
trie = Trie()
trie.insert_all(archive['trajs'], [None for _ in range(len(archive['trajs']))])

In [16]:
# np.save('temp_trie.npy', trie)
# np.save('temp_arr.npy', archive['trajs'])

In [32]:
archive.keys()

dict_keys(['trajs', 'rets', 'scores', 'config'])

In [33]:
sum([len(i) for i in archive['trajs']])/1e6

17.614785

In [45]:
def trie_num_nodes(root):
    assert root is not None
    count = 0
    stack = [root]
    while stack:
        node = stack.pop()
        count += 1
        stack.extend(node.children.values())
    return count

In [47]:
trie_num_nodes(root)/1e6

0.108528

In [1]:
import torch

In [2]:
A = torch.randn(128, 16, 32)
B = torch.randn(8, 32)

SyntaxError: incomplete input (3135128653.py, line 1)